In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
import json
import datetime

# 1. GIỚI THIỆU & CÂU HỎI

YouTube (trang web chia sẻ video nổi tiếng thế giới) duy trì danh sách các video thịnh hành nhất trên nền tảng này. Theo tạp chí Variety, “Để xác định video thịnh hành nhất của năm, YouTube sử dụng kết hợp nhiều yếu tố bao gồm đo lường tương tác của người dùng (số lượt xem, lượt chia sẻ, nhận xét và lượt thích). 

Xu hướng giúp người xem biết những gì đang diễn ra trên YouTube và trên thế giới. Xu hướng nhằm hiển thị những video mà nhiều người xem sẽ thấy thú vị. Một số xu hướng có thể dự đoán được, chẳng hạn như bài hát mới của một nghệ sĩ nổi tiếng hoặc đoạn giới thiệu phim mới. Những người khác gây ngạc nhiên, giống như một video lan truyền. Xu hướng không được cá nhân hóa và hiển thị cùng một danh sách các video thịnh hành ở mỗi quốc gia cho tất cả người dùng.

Danh sách các video thịnh hành được cập nhật khoảng 15 phút một lần. Với mỗi lần cập nhật, video có thể di chuyển lên, xuống hoặc giữ nguyên vị trí trong danh sách. Những nghệ sĩ biểu diễn hàng đầu trong danh sách thịnh hành của YouTube là các video ca nhạc (chẳng hạn như "Gangam Style" nổi tiếng), người nổi tiếng và / hoặc các buổi biểu diễn truyền hình thực tế và các video lan truyền ngẫu nhiên của anh chàng có máy ảnh mà YouTube nổi tiếng.

## Câu hỏi:
1. Trending với lượt view: Có bao nhiêu lượt xem khi 1 video lọt top trending?
2. Trending videos titles:
    + Những từ phổ biến trong các title?
    + 2-Gam phổ biến nhất trong title.
3. Video xu hướng được xuất bản khi nào, bao lâu thì video trở thành video xu hướng?
4. Lượt xem, lượt thích, lượt không thích, số lượt nhận xét, độ dài tiêu đề và các thuộc tính khác có tương quan (liên quan đến) nhau như thế nào? Chúng được kết nối với nhau như thế nào?
5. Danh mục video nào (ví dụ: Giải trí, Trò chơi, Hài, v.v.) có số lượng video thịnh hành nhất?
6. Mô tả của video xu hướng:
    + Độ dài của mô tả?
    + Những từ ngữ phổ biến trong phần mô tả này?
    + Video thịnh hành có chứa hashtags trong phần mô tả không?


# 2. ĐỌC DỮ LIỆU

In [2]:
data_df = pd.read_csv("USvideos.csv")

##  Thông tin về dữ liệu

Tập dữ liệu này là bản ghi hàng ngày về các video thịnh hành nhất trên YouTube (năm 2017-2018) mà sử dụng cho lab này là thống kê của US. Được tác giả thu thấp thông qua Youtube API.

Link sưu tầm: https://www.kaggle.com/datasnaek/youtube-new
+ Usage Information
    - License: CC0 Public Domain
    - Visibility:       Public
+ Maintainers
    - Dataset owner: Mitchell J
+ Updates
    - Expected update frequency: Not specified
    - Last updated: 2019-06-03
    - Date created: 2017-11-14
    - Current version: Version 115

# 3. KHÁM PHÁ DỮ LIỆU

In [3]:
data_df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


## Kích thước dữ liệu

In [4]:
data_df.shape

(40949, 16)

+ Số dòng: 40949
+ Số cột: 16

## Số lượng video là unique

In [5]:
data_df['video_id'].unique().shape[0]

6351

## Ý nghĩa của mỗi thuộc tính:
+ video_id: biến định danh video
+ trending_date: ngày video lên top trending
+ title: tựa đề của video
+ channel_title: kênh đăng tải video lên Youtube
+ category_id: phân loại id, mỗi nước sẽ có biến phân loại này khác nhau
+ publish_time: ngày video được push lên Youtube
+ tags: Khi tải video lên YouTube, bạn có thể thêm thẻ vào video. Thẻ không được hiển thị trên trang video; bạn có thể thấy chúng bằng cách xem mã nguồn của trang hoặc bằng cách sử dụng các tiện ích mở rộng của trình duyệt như TubeBuddy và vidIQ.
+ views: lượng xem tính đến ngày lên top
+ likes: lượng thích tính đến ngày lên top
+ dislikes: lượng không tính đến ngày lên top
+ comment_count: số bình luận
+ thumbnail_link: link hình ảnh nhỏ của video khi truy cập video, đóng một vai trò quan trọng trong sự thành công của video YouTube vì chúng ảnh hưởng đến tỷ lệ nhấp (số lượng người nhấp vào video của bạn khi họ xem)
+ comments_disabled: ? 
+ ratings_disabled: ?
+ video_error_or_removed: video có lỗi hay bị xóa đi(True nếu có lỗi hoặc bị xóa và ngược lại)
+ description: phần mô tả cho bài đăng

## Mỗi cột có dtype là gì:

In [6]:
col_dtypes=pd.Series(data_df.dtypes,index=data_df.columns)
col_dtypes

video_id                  object
trending_date             object
title                     object
channel_title             object
category_id                int64
publish_time              object
tags                      object
views                      int64
likes                      int64
dislikes                   int64
comment_count              int64
thumbnail_link            object
comments_disabled           bool
ratings_disabled            bool
video_error_or_removed      bool
description               object
dtype: object

## Cột có dtype là object nghĩa là sao?

## Miền giá trị của các cột dạng số?

In [7]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40949 entries, 0 to 40948
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   video_id                40949 non-null  object
 1   trending_date           40949 non-null  object
 2   title                   40949 non-null  object
 3   channel_title           40949 non-null  object
 4   category_id             40949 non-null  int64 
 5   publish_time            40949 non-null  object
 6   tags                    40949 non-null  object
 7   views                   40949 non-null  int64 
 8   likes                   40949 non-null  int64 
 9   dislikes                40949 non-null  int64 
 10  comment_count           40949 non-null  int64 
 11  thumbnail_link          40949 non-null  object
 12  comments_disabled       40949 non-null  bool  
 13  ratings_disabled        40949 non-null  bool  
 14  video_error_or_removed  40949 non-null  bool  
 15  de

Chúng ta có thể thấy rằng có 40,949 mục nhập trong tập dữ liệu. Chúng ta cũng có thể thấy rằng tất cả các cột trong tập dữ liệu đều hoàn chỉnh (tức là chúng có 40.949 mục nhập không phải null) ngoại trừ cột mô tả có một số giá trị rỗng; nó chỉ có 40.379 giá trị không rỗng.

In [8]:
data_df.describe()

,category_id,views,likes,dislikes,comment_count
count,40949.000000,4.094900e+04,4.094900e+04,4.094900e+04,4.094900e+04
mean,19.972429,2.360785e+06,7.426670e+04,3.711401e+03,8.446804e+03
std,7.568327,7.394114e+06,2.288853e+05,2.902971e+04,3.743049e+04
min,1.000000,5.490000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,17.000000,2.423290e+05,5.424000e+03,2.020000e+02,6.140000e+02
50%,24.000000,6.818610e+05,1.809100e+04,6.310000e+02,1.856000e+03
75%,25.000000,1.823157e+06,5.541700e+04,1.938000e+03,5.755000e+03
max,43.000000,2.252119e+08,5.613827e+06,1.674420e+06,1.361580e+06


+ Số lượt xem trung bình của một video thịnh hành là 2.360.784. Giá trị trung bình của số lượt xem là 681.861, có nghĩa là một nửa số video thịnh hành có lượt xem nhỏ hơn con số đó và nửa còn lại có lượt xem lớn hơn con số đó
+ Số lượt thích trung bình của một video thịnh hành là 74.266, trong khi số lượt không thích trung bình là 3.711. 
+ Số bình luận trung bình là 8.446 trong khi trung bình là 1.856

## Năm thu thập dữ liệu

In [9]:
data_df["trending_date"].apply(lambda x: '20' + x[:2]).value_counts(normalize=True)

2018    0.765562
2017    0.234438
Name: trending_date, dtype: float64

Chúng ta có thể thấy rằng tập dữ liệu được thu thập vào năm 2017 và 2018. Trong đó năm 2018 là chủ yếu.

# 4. TIỀN XỬ LÍ

## Cột mô tả có một số giá trị rỗng. Chúng ta có thể thấy rằng các giá trị null được ký hiệu là NaN

In [10]:
data_df["description"] = data_df["description"].fillna(value="")

## Truy xuất danh muc/thể loại của các video mà theo tiêu chuẩn ở US, thay thế cột 'categoryId' thành 'category'.

Danh mục phận loại này được cung cấp chung với dữ liệu: US_category_id.json

In [11]:
#Tham khảo code: https://www.datacamp.com/community/tutorials/json-data-python?utm_source=adwords_ppc&utm_campaignid=1455363063&utm_adgroupid=65083631748&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=&utm_creative=332602034358&utm_targetid=aud-299261629574:dsa-429603003980&utm_loc_interest_ms=&utm_loc_physical_ms=1028581&gclid=Cj0KCQjws536BRDTARIsANeUZ59jJ9yzfjnsGBS0N-Vrfgo0X310RzopRJcnzPN3rkJArMzehAu7Ty8aAvkNEALw_wcB
with open("US_category_id.json") as f:
    categories = json.load(f)["items"]
    print(categories)
cat_dict = {}
for cat in categories:
    cat_dict[int(cat["id"])] = cat["snippet"]["title"]#Từ id truy xuất đến title(trong snippet) là thể loại video.
data_df['category_name'] = data_df['category_id'].map(cat_dict)#Tạo cột mới

[{'kind': 'youtube#videoCategory', 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKmPBggty2mZQ"', 'id': '1', 'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'title': 'Film & Animation', 'assignable': True}}, {'kind': 'youtube#videoCategory', 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45ZTFR3a3NyTA"', 'id': '2', 'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'title': 'Autos & Vehicles', 'assignable': True}}, {'kind': 'youtube#videoCategory', 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxbknKFVe5Lmg"', 'id': '10', 'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'title': 'Music', 'assignable': True}}, {'kind': 'youtube#videoCategory', 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-oBJavSGkfDI"', 'id': '15', 'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'title': 'Pets & Animals', 'assignable': True}}, {'kind': 'youtube#videoCategory', 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/9GQMSRjrZdHeb1OEM1XVQ9zbGec"', 'id': '17', 'snippet': {'channelId'

## Ta thấy cột publish_time chứa cả thời gian ngày vào giờ đăng video. Ta cần tách date và time thành 2 cột mới cho dữ liệu này.

In [12]:
data_df['publish_time'] = pd.to_datetime(data_df['publish_time'])

In [13]:
data_df["publishing_day"] = data_df["publish_time"].dt.day_name()
data_df["publishing_hour"] = data_df["publish_time"].dt.hour

## Chuyển giá trị trending_date thành đúng định dạng datetime

In [14]:
data_df['trending_date'] = pd.to_datetime(data_df['trending_date'], format='%y.%d.%m')

## Drop một số cột không cần thiết

In [15]:
data_df.drop(['category_id','publish_time','thumbnail_link','comments_disabled','ratings_disabled','video_error_or_removed'], axis=1, inplace=True)

In [16]:
data_df.head(1).transpose()

,0
video_id,2kyS6SvSYSE
trending_date,2017-11-14 00:00:00
title,WE WANT TO TALK ABOUT OUR MARRIAGE
channel_title,CaseyNeistat
tags,SHANtell martin
views,748374
likes,57527
dislikes,2966
comment_count,15954
description,SHANTELL'S CHANNEL - https://www.youtube.com/s...


# Phân tích dữ liệu để trả lời các câu hỏi 